In [43]:
import pandas
import requests
import xml.etree.ElementTree as ET

# Adresa API - jde otevřít normálně v prohlížeči
response = requests.get('https://api.aiscr.cz/dapro/oai?verb=ListRecords&metadataPrefix=oai_amcr&set=akce&metadataPrefix=oai_amcr')
# Úprava kódování
response.encoding = "utf-8"

In [21]:
# Uložení dat do souboru - pouze pro kontrolu
with open('list_records.xml', 'w', encoding="utf-8") as f:
    print(response.text, file=f)

Toto je nejsložitější část. Data jsou ve formátu XML, který je založený na používání párových značek. Například hodnota `vedouci_akce` je vložena do párových značek takto: `<vedouci_akce>Kos, Petr</vedouci_akce>`. Celá akce je pak uložená v párové značce `<akce></akce>`. My chceme načíst všechny párové značky `akce`, což za nás provede parametr `xpath=".//akce`. Použitý je vyhledávají jazyk XPath, ale to řešit nemusíte, s ním už jinde asi pracovat nebudeme.

Data jsou načtena do tabulky modulu `pandas`. Práci s modulem `pandas` budete probírat v rámci předmětu Python Data. Funguje podobně jako SQL, takže můžete vybírat konkrétní sloupečky, filtrovat řádky atd.

![structure.PNG](assets/structure.PNG)

In [42]:
df = pandas.read_xml(response.text, xpath=".//akce", parser="etree")
# Metoda set_index nastaví index, který v pandas funguje podobně jako v databázích. Nastavíme jako index sloupec ident_cely
df = df.set_index("ident_cely")
# Hlavička - podívám se na prvních pět řádku, jestli se data načetla v pořádku
df.head()

,typ,stav,je_nz,okres,katastr,vedouci_akce,organizace,uzivatelske_oznaceni,specifikace_data,datum_zahajeni,...,child_dok_jednotka,datum_podani_nz,odpovedny_pracovnik_podani_nz,datum_archivace,odpovedny_pracovnik_archivace,vazba_projekt_akce,child_dokument,dalsi_katastry,vedouci_akce_ostatni,organizace_ostatni
ident_cely,,,,,,,,,,,,,,,,,,,,,
M-9102504A,N,5,0,BŘECLAV,HUSTOPEČE U BRNA,"Kos, Petr",ÚAPP Brno,143/2004,přesně,0200-09-08,...,M-9102504A-D01,None,None,None,None,NaN,None,None,None,None
M-9105056A,N,8,1,OPAVA,SLAVKOV U OPAVY,"Malík, Petr",NPÚ ú.o.p. Ostrava,062/2011,přesně,0201-08-09,...,M-9105056A-D01,2023-01-18,"Anonym, ARÚB",2023-01-18,"Anonym, ARÚB",NaN,None,None,None,None
M-9106217A,N,8,1,OPAVA,VÍTKOV,"Krasnokutská, Tereza",NPÚ ú.o.p. Ostrava,098/2016,přesně,0201-09-19,...,M-9106217A-D01,2023-02-01,"Anonym, ARÚB",2023-02-01,"Anonym, ARÚB",NaN,None,None,None,None
M-9106246A,N,8,1,OPAVA,ŠTĚPÁNKOVICE,"Krasnokutská, Tereza",NPÚ ú.o.p. Ostrava,065/2016,přesně,0206-06-19,...,M-9106246A-D01,2023-01-25,"Anonym, ARÚB",2023-01-25,"Anonym, ARÚB",NaN,None,None,None,None
C-201663968A,R,8,0,PLZEŇ-SEVER,KRALOVICE U RAKOVNÍKA,"Stráník, Daniel",Muzeum Mariánská Týnice,"AR 8/2016 - Kralovice, Za Rájem, výstavba RD",přesně,0216-05-01,...,C-201663968A-D01,2022-02-21,"Stráník, Daniel",2022-07-14,"Švejcar, Ondřej",NaN,C-TX-202200118,None,None,None


Tady používáme jmenné prostory (name spaces). Ve jmených prostorech jsou definovány různé značky. My nyní potřebujeme tag `resumptionToken` ("token pro pokračování"), který řeší "pokračování" v požadavcích. Najednou je možné stáhnout pouze 1000 záznamů. Pro dalších 1000 záznamů musíme API využít znovu a dodat mu tento pokračovací token, aby API vědělo, které záznamy už nám poskytlo a které jsou nyní na řadě.

In [39]:
ns = {'oai': 'http://www.openarchives.org/OAI/2.0/'}

root = ET.fromstring(response.text)
resumptionToken = root.find('.//oai:', ns).text
print(resumptionToken)

2023-03-24.oai_amcr.1000.akce.1000.1000...t


Zde je testovací uložení do CSV. Uvnitř hranatých závorek jsou pole (sloupce), které chceme uložit. Nám bude stačit pouze část dat, takže si vybereme ty sloupce, které nás zajímají.

In [41]:
df_csv = df[["ident_cely", "vedouci_akce", "organizace", "katastr", "okres", "datum_zahajeni", "datum_ukonceni"]]
df_csv.to_csv("export.csv")